In [6]:
import json
from tqdm import tqdm
from nostr_neo4j import utils
from nostr_neo4j import Event

In [5]:
with open('data/events.jsonl') as f:
    events = [json.loads(line) for line in f]

In [3]:
event = events[-1]
event

{'id': 'e0c4411dd0ec710c3f980c0eecbeb7f6575204487e896d8717391c4bddb745bd',
 'created_at': 1684451725,
 'kind': 0,
 'pubkey': '7556987aa829381934ac6e8d9b71a4da304507f05a7ad60e447bd48218fd6faf',
 'sig': '9a6f6428cdc155052fffb71c71cc0955047998ea5b485b851c13da54663c25b43cd395d8f9dc8f8cce238f86b7b1c955ba62b0f73c598d1879a5b5de4dbc3ff9',
 'content': '{\\"display_name\\":\\"限时领券优惠\\",\\"website\\":\\"https://55uu.wang\\",\\"name\\":\\"京东拼多多抖音淘宝优惠券\\", \\"about\\":\\"京东,拼多多,抖音,淘宝,限时领券优惠\\", \\"picture\\":\\"https://tftc.io/content/images/2023/01/Screen-Shot-2023-01-30-at-11.11.27-PM.png\\",\\"nip05\\":\\"npub1w4tfs74g9yupjd9vd6xekudymgcy2plstfadvrjy002gyx8ad7hsj782lv@nostr-vip.top\\"}',
 'tags': []}

In [4]:
to_fix = {
    'wrong_sig': [],
    'wrong_id': [],
}
for index, event in tqdm(enumerate(events), total=len(events)):
    event_id = utils.calc_event_id(
        event['pubkey'],
        event['created_at'],
        event['kind'],
        event['tags'],
        event['content']
    )
    sigs_verified = utils.verify_signature(event['id'], event['pubkey'], event['sig'])
    if not sigs_verified:
        to_fix['wrong_sig'].append(index)
    if event_id != event['id']:
        to_fix['wrong_id'].append(index)

100%|██████████| 10000000/10000000 [06:39<00:00, 25010.84it/s]


In [2]:
# import mongodb

import pymongo

In [7]:
# connect to the MongoDB enets database if not exist it will be created
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["events"]
collection = db["events"]

In [8]:
for event in events:
    event = Event.from_dict(event)
    collection.insert_one(event.to_dict())
    
